In [1]:
# Coverting csv data files into a sqlite database
# By Chris Chiang
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, Column
from sqlalchemy.types import Float, String, Date, BigInteger
from sqlalchemy.ext.declarative import declarative_base

In [9]:
# load in csv file and some cleaning and sorting
data_loc = '../cleaned_data/Combine_data_tables/'
US_df = pd.read_csv(data_loc + 'US_ALL.csv')
It_df = pd.read_csv(data_loc + 'It_ALL.csv')
KR_df = pd.read_csv(data_loc + 'KR_ALL.csv')
CN_df = pd.read_csv(data_loc + 'CN_ALL.csv')
tot_df = pd.read_csv(data_loc + 'Total.csv')
event_df = pd.read_csv(data_loc + 'article_new.csv')
event_df=event_df.rename(columns={'date':'Date','article':'Major Event'})
US_df.sort_values('Date',inplace=True)
It_df.sort_values('Date',inplace=True)
KR_df.sort_values('Date',inplace=True)
CN_df.sort_values('Date',inplace=True)
tot_df.sort_values('Date',inplace=True)
event_df.sort_values('Date',inplace=True)

In [10]:
US_df['Date']=US_df['Date'].apply(lambda x : dt.date.fromisoformat(x))
It_df['Date']=It_df['Date'].apply(lambda x : dt.date.fromisoformat(x))
CN_df['Date']=CN_df['Date'].apply(lambda x : dt.date.fromisoformat(x))
KR_df['Date']=KR_df['Date'].apply(lambda x : dt.date.fromisoformat(x))
tot_df['Date']=tot_df['Date'].apply(lambda x : dt.date.fromisoformat(x))
event_df['Date']=event_df['Date'].apply(lambda x : dt.date.fromisoformat(x))

In [11]:
KR_df

,Date,Country/Region,Confirmed cases,Deaths,Index Price,Index Daily % Change
64,2019-12-23,South Korea,0.0,0.0,2203.71,-0.02
63,2019-12-24,South Korea,0.0,0.0,2190.08,-0.62
62,2019-12-26,South Korea,0.0,0.0,2197.93,0.36
61,2019-12-27,South Korea,0.0,0.0,2204.21,0.29
60,2019-12-30,South Korea,0.0,0.0,2197.67,-0.30
...,...,...,...,...,...,...
4,2020-03-23,South Korea,8961.0,111.0,1482.46,-5.34
3,2020-03-24,South Korea,9037.0,120.0,1609.97,8.60
2,2020-03-25,South Korea,9137.0,126.0,1704.76,5.89
1,2020-03-26,South Korea,9241.0,131.0,1686.24,-1.09


In [8]:
# create sqlite file
engine = create_engine("sqlite:///Covid19DB_VS_Market.sqlite", echo=True)
conn = engine.connect()

2020-03-30 20:35:04,148 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-30 20:35:04,149 INFO sqlalchemy.engine.base.Engine ()
2020-03-30 20:35:04,149 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-30 20:35:04,150 INFO sqlalchemy.engine.base.Engine ()


In [12]:
# define data types
table_type_country={'Date':Date ,'Country/Region':String,'Confirmed cases':BigInteger,'Deaths':BigInteger,'Index Price':Float,'Index Daily % Change':Float}
table_type_tot={'Date':Date ,'Country/Region':String,'Confirmed cases':BigInteger,'Deaths':BigInteger}
table_type_eve={'Date':Date ,'Major Event':String}

In [23]:
# load data frames into sqlite
US_df.to_sql('US_data', conn,index = False, dtype =table_type_country,if_exists='replace')
It_df.to_sql('Italy_data', conn,index = False, dtype =table_type_country,if_exists='replace')
CN_df.to_sql('China_data', conn,index = False, dtype =table_type_country,if_exists='replace')
KR_df.to_sql('Korea_data', conn,index = False, dtype =table_type_country,if_exists='replace')
tot_df.to_sql('Total_table', conn,index = False, dtype =table_type_tot,if_exists='replace')
event_df.to_sql('Event_table', conn,index = False, dtype =table_type_tot,if_exists='replace')



e.base.Engine PRAGMA temp.index_list("Italy_data")
2020-03-30 20:47:02,660 INFO sqlalchemy.engine.base.Engine ()
2020-03-30 20:47:02,661 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Italy_data' AND type = 'table'
2020-03-30 20:47:02,661 INFO sqlalchemy.engine.base.Engine ()
2020-03-30 20:47:02,662 INFO sqlalchemy.engine.base.Engine 
DROP TABLE "Italy_data"
2020-03-30 20:47:02,662 INFO sqlalchemy.engine.base.Engine ()
2020-03-30 20:47:02,666 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-30 20:47:02,667 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Italy_data" (
	"Date" DATE, 
	"Country/Region" VARCHAR, 
	"Confirmed cases" BIGINT, 
	"Deaths" BIGINT, 
	"Index Price" FLOAT, 
	"Index Daily % Change" FLOAT
)


2020-03-30 20:47:02,668 INFO sqlalchemy.engine.base.Engine ()
2020-03-30 20:47:02,672 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-30 20:47:02,672 INFO sqlalchemy.engine.base.En

In [24]:
engine.execute('select * from Event_table').fetchall()


2020-03-30 20:47:11,269 INFO sqlalchemy.engine.base.Engine select * from Event_table
2020-03-30 20:47:11,270 INFO sqlalchemy.engine.base.Engine ()


[('2019-12-31', 'December 31, 2019 -Cases of pneumonia detected in Wuhan, China,are first reported to the WHO.During this reported period, the virus is unknown. The casesoccur between December 12 and December 29,according to Wuhan Municipal Health.'),
 ('2020-01-01', 'January 1, 2020 -Chinese health authorities close theHuanan Seafood Wholesale Marketafter it is discovered that wild animals sold there may be the source of the virus.'),
 ('2020-01-05', 'January 5, 2020 -China announces that the unknown pneumonia cases in Wuhan are not SARS or MERS. In astatement,the Wuhan Municipal Health Commission says a retrospective probe into the outbreak has been initiated.'),
 ('2020-01-07', 'January 7, 2020 -Chinese authorities confirm that they have identified the virus as a novel coronavirus, initially named2019-nCoVby the WHO.'),
 ('2020-01-11', 'January 11, 2020 -The Wuhan Municipal Health Commission announces the first death caused by the coronavirus. A 61-year-old man, exposed to the virus

In [54]:
US_df["Date"].value_counts()

2020-02-04    1
2020-02-05    1
2020-02-03    1
2020-03-02    1
2020-01-06    1
             ..
2020-01-14    1
2019-12-30    1
2020-02-21    1
2020-01-17    1
2020-03-26    1
Name: Date, Length: 66, dtype: int64